En este notebook se hara el proceso de **PRIORIZAR** , se tomaran las PQR Desde diciembre del 2022 a  **2023 a Marzo(periodo de tres meses)**


# CONFIGURAR SPARK

In [ ]:
import pandas as pd
import sys
sys.path.append('../librerias')


# LIBRERIAS





In [ ]:
!python -m spacy download es_core_news_sm

!python -m spacy download es_core_news_lg

In [2]:
import pandas as pd

from google.colab import drive
import gc

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification




nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing


#.master("local[2]")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

# ETL

In [ ]:

pandas_c_F= pd.read_csv('/content/PRIORIZAR/entranamiento_/binario_entrenamiento_cruda.csv',delimiter=';')

pandas_c_F= pandas_c_F.fillna("vacios")


pandas_c_F

In [ ]:
pandas_c_F.isnull().sum()

In [ ]:
from clean_text import clean_text

In [ ]:
#declare answers and questions
descripcion=pandas_c_F.DESCRIPCION_PQR	



# Cleaning the questions

clean_descripcion = []
for des in descripcion:
    clean_descripcion.append(clean_text(des))


#restar dos para optener la data de excel y poder comparar

clean_descripcion  [1002]

In [ ]:
len(clean_descripcion)

In [ ]:
from tqdm import tqdm
from spelling import correct_sentence

In [ ]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

In [ ]:
corrected_sentences1=corrected_sentences

In [4]:
from lemastem import data_preprocessing


clean_descripcion=data_preprocessing(corrected_sentences1)


clean_descripcion[883]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


11659


In [ ]:
clean_descripcion[0]

In [ ]:
import pickle



# Specify the file path where you want to save the list
file_path = "/content/PRIORIZAR/data/entrenamiento_binario.pkl"

# Open the file in binary write mode
with open(file_path, "wb") as file:
    # Use the pickle.dump() function to save the list to the file
    pickle.dump(clean_descripcion, file)

In [ ]:
import pickle

# Specify the file path from where you want to load the list
file_path = "/content/PRIORIZAR/entranamiento_/entrenamiento_binario_final.pkl"

# Open the file in binary read mode
with open(file_path, "rb") as file:
    # Use the pickle.load() function to load the list from the file
    loaded_list = pickle.load(file)

print(loaded_list)

In [ ]:

clean_descripcion=loaded_list

In [ ]:
len(clean_descripcion)

In [ ]:
clean_descripcion[5668]

creacion tabla

In [ ]:
import numpy as np
import pandas as pd
from unidecode import unidecode



merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion']).reset_index()

#############################################

pandas_c_F.reset_index(inplace=True)

#merged_df = pd.merge(merged_df, pandas_c_F, on='index', how='inner').loc[:, ['FECHA_CREACION','NUMERO_PQR','descripcion', 'clasificacion']]

merged_df = pd.concat([merged_df, pandas_c_F], axis=1)



shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)




#shuffled_df.rename(columns={"FECHA_CREACION": "event_timestamp"}, inplace=True)

#string_values = ['abcde', 'fghij', 'klmno', 'pqrst', 'uvwxy', 'zabcd', 'efghi', 'jklmn', 'opqrs', 'tuvwx']



# Función para eliminar las tildes de un texto
def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
shuffled_df['descripcion'] = shuffled_df['descripcion'].apply(remove_accents)



# Randomly select string values to fill the DataFrame
#random_strings = np.random.choice(string_values, size=len(shuffled_df))

#shuffled_df["NUMERO_PQR"]=random_strings


shuffled_df

In [ ]:
shuffled_df.to_csv('/content/PRIORIZAR/entranamiento_/entrana_bina.csv', index=False)

In [ ]:
shuffled_df = pd.read_csv('/content/PRIORIZAR/entranamiento_/entrana_bina.csv')

In [ ]:
from tfidf import TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from joblib import dump



X = shuffled_df['descripcion']

y = shuffled_df['clasificacion']




# train test split (66% train - 33% test)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)




X_train,X_test = TFIDF(X_train,X_test)

pca = PCA()

pca.fit(X_train)

# Get the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Find the number of components that explain a desired amount of variance
desired_variance = 0.9  # Set the desired variance threshold
n_components = np.argmax(cumulative_variance_ratio >= desired_variance) + 1

# Create a new instance of PCA with the optimal number of components
pca = PCA(n_components=n_components)


X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)



print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)



print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)


dump(value=pca, filename="/content/PRIORIZAR/modelo/pca_binario.joblib")

In [ ]:
n_components

In [ ]:
pip install optuna

In [ ]:
import optuna
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score


In [ ]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimators": trial.suggest_int("n_estimators", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 5.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 30, 90),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'learning_rate': trial.suggest_float("learning_rate", 0.04, 0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }




    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train_new, y_train)
    preds = gbm.predict(X_test_new)
    accuracy = accuracy_score(y_test, preds)
    return accuracy




In [ ]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

In [ ]:
print('Best parameters:', study.best_params)

In [ ]:
mode = lgb.LGBMClassifier(**study.best_params)
mode.fit(X_train_new, y_train)

In [ ]:
y_pred = mode.predict(X_test_new)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

report = classification_report(y_test, y_pred)
print(report)

In [ ]:
from joblib import dump
dump(value=mode, filename="/content/PRIORIZAR/modelo/modelo_binario.joblib")

para data a predecir

modelo creacion

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA



X = shuffled_df['descripcion']

y = shuffled_df['clasificacion'].astype(int)




# train test split (66% train - 33% test)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)






def TFIDF(X_train, X_test, MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with", str(np.array(X_train).shape[1]), "features")
    return (X_train, X_test)


X_train,X_test = TFIDF(X_train,X_test)



pca = PCA(n_components=5000)
X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)





print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

lr = LogisticRegression()



lr.fit(X_train_new, y_train)



# generate predictions

predictions = lr.predict(X_test_new)

predictions

In [ ]:
import lightgbm as lgb
from joblib import dump

train_data = lgb.Dataset(X_train_new, label=y_train)

# Set hyperparameters for LightGBM
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the LightGBM model
model = lgb.train(params, train_data, num_boost_round=100)

# Make predictions on the test set
y_pred = model.predict(X_test_new)

y_pred_binary = [1 if val >= 0.5 else 0 for val in y_pred]

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)


from sklearn import metrics

df = pd.DataFrame(metrics.confusion_matrix(y_test,y_pred_binary), index=['OK','WRONG'], columns=['OK','WRONG'])

df

In [ ]:
dump(value=model, filename="model.joblib")

predicciendo data nueva

In [ ]:
store = FeatureStore(repo_path="/my_project/feature_repo/")

t_predict = store.get_saved_dataset(name="prediccion_2").to_df()

t_predict

In [ ]:
X_train = merged_df['descripcion']





def TFIDF(X_train, X_test, MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with", str(np.array(X_train).shape[1]), "features")
    return (X_train, X_test)


X_train,X_test = TFIDF(X_train,_)





pca = PCA(n_components=5000)
X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)



print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)

In [ ]:
y_pred = model.predict(X_train_new)

y_pred_binary = [1 if val >= 0.5 else 0 for val in y_pred]


In [ ]:
y_pred_binary

In [ ]:

pandas_c_F['prediccion_algoritmo'] = y_pred_binary

pandas_c_F

In [ ]:
pandas_c_F.to_csv('c_facturados.csv', index=False)

In [ ]:
from feast import FeatureStore
from datetime import datetime, timedelta

# Getting our FeatureStore
store = FeatureStore(repo_path="/my_project/feature_repo/")

# Code for loading features to online store between two dates
"""store.materialize(
    end_date=datetime.now(),
    start_date=datetime.now() - timedelta(days=900))"""

# Loading the latest features after a previous materialize call or from the beginning of time
#store.materialize_incremental(end_date=datetime.now())


store.materialize(
    end_date=datetime.now(),
    start_date=partida)

In [ ]:
from datetime import datetime, timedelta



partida=datetime.now() - timedelta(days=60)

# Print the future date
print(f"Current Date: {partida}")


In [ ]:
feast_features = [
        "prediccion:descripcion"
    ]


In [ ]:
features = store.get_online_features(
    features=feast_features,    
    entity_rows=[{"NUMERO_PQR": "abcde"}]
).to_dict()

string_values = ['abcde', 'fghij', 'klmno', 'pqrst', 'uvwxy', 'zabcd', 'efghi', 'jklmn', 'opqrs', 'tuvwx']

In [ ]:
features

In [ ]:
features_df = pd.DataFrame.from_dict(data=features)

features_df